In [2]:
from bs4 import BeautifulSoup
import requests

URL = 'https://indiankanoon.org/browse/'
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')

# Find all links
all_links = soup.find_all('a')
court_links = {}
for link in all_links:
    name = link.text.strip()
    href = link.get('href', '')
    if 'supreme court' in name.lower():
        court_links[name] = href

print(court_links)  # Should show 'Supreme Court of India' and its URL

{'Supreme Court of India': '/browse/supremecourt/', 'Supreme Court - Daily Orders': '/browse/scorders/'}


In [3]:
#Below code is used to get the browse url of supreme court of india

import requests
from bs4 import BeautifulSoup
import time
import json

base = 'https://indiankanoon.org'
start_url = base + '/browse/'
page = requests.get(start_url)
soup = BeautifulSoup(page.content, 'html.parser')

# Find the 'Supreme Court of India' link
supreme_link = None
for a in soup.find_all('a'):
    if a.text.strip() == 'Supreme Court of India':
        supreme_link = base + a['href']
        break

if not supreme_link:
    raise Exception("Supreme Court link not found")

print("Supreme Court browse URL:", supreme_link)


Supreme Court browse URL: https://indiankanoon.org/browse/supremecourt/


In [4]:
#gets the links of all the years found in supreme court sectionn

#page = requests.get(supreme_link)
#soup = BeautifulSoup(page.content, 'html.parser')

#years = []
soup = BeautifulSoup(requests.get('https://indiankanoon.org/browse/supremecourt/').content, 'html.parser')

# Find all year links (they are under class 'browselist')
year_links = []
for div in soup.find_all('div',class_='browselist'):
    for a in div.find_all('a'):
        year_links.append(('https://indiankanoon.org' + a['href'], a.text.strip()))
        '''year_text = a.text.strip()
        # Only keep years in reasonable range
        if year_text.isdigit() and 1947 <= int(year_text) <= 2020:
            year_links.append((year_text, base + a['href']))'''

print("Year links found:", year_links[:3], "...")  # Show a few as sample


Year links found: [('https://indiankanoon.org/browse/supremecourt/1950/', '1950'), ('https://indiankanoon.org/browse/supremecourt/1951/', '1951'), ('https://indiankanoon.org/browse/supremecourt/1952/', '1952')] ...


In [5]:
#for each year scrap months

months = []
for year_url, year_text in year_links:
    soup = BeautifulSoup(requests.get(year_url).content, 'html.parser')
    for div in soup.find_all('div', class_='browselist'):
        for a in div.find_all('a'):
            months.append(('https://indiankanoon.org' + a['href'], a.text.strip(), year_text))

print(months)


[('https://indiankanoon.org/search/?formInput=doctypes:supremecourt fromdate:1-1-1950 todate: 31-1-1950', 'January', '1950'), ('https://indiankanoon.org/search/?formInput=doctypes:supremecourt fromdate:1-2-1950 todate: 28-2-1950', 'February', '1950'), ('https://indiankanoon.org/search/?formInput=doctypes:supremecourt fromdate:1-3-1950 todate: 31-3-1950', 'March', '1950'), ('https://indiankanoon.org/search/?formInput=doctypes:supremecourt fromdate:1-4-1950 todate: 30-4-1950', 'April', '1950'), ('https://indiankanoon.org/search/?formInput=doctypes:supremecourt fromdate:1-5-1950 todate: 31-5-1950', 'May', '1950'), ('https://indiankanoon.org/search/?formInput=doctypes:supremecourt fromdate:1-6-1950 todate: 30-6-1950', 'June', '1950'), ('https://indiankanoon.org/search/?formInput=doctypes:supremecourt fromdate:1-7-1950 todate: 31-7-1950', 'July', '1950'), ('https://indiankanoon.org/search/?formInput=doctypes:supremecourt fromdate:1-8-1950 todate: 31-8-1950', 'August', '1950'), ('https://ind

In [7]:
import requests
from bs4 import BeautifulSoup
import time
import json
from urllib.parse import urljoin

all_judgment_links = []

for month_url, month_name, year_text in months:
    # Remove any accidental extra spaces in the URL
    month_url = month_url.replace('todate: ', 'todate:').replace('fromdate: ', 'fromdate:')
    print(f"Processing {month_name} {year_text} ...")
    for pagenum in range(100):  # Adjust as needed
        url = f"{month_url}&pagenum={pagenum}"
        print("Fetching:", url)  # Debug
        r = requests.get(url)
        soup = BeautifulSoup(r.content, 'html.parser')
        results = soup.find_all('a', class_='result_title')
        print("Results found on page:", len(results))  # Debug
        if not results:
            break
        for result in results:
            # Use urljoin to handle relative URLs robustly
            judgment_url = urljoin('https://indiankanoon.org', result['href'])
            all_judgment_links.append(judgment_url)
        time.sleep(1)
        print("Total collected so far:", len(all_judgment_links))

with open("supreme_court_judgments.json", "w") as f:
    json.dump(all_judgment_links, f, indent=2)
print(f"Total judgments scraped: {len(all_judgment_links)}")



Processing January 1950 ...
Fetching: https://indiankanoon.org/search/?formInput=doctypes:supremecourt fromdate:1-1-1950 todate:31-1-1950&pagenum=0
Results found on page: 0
Processing February 1950 ...
Fetching: https://indiankanoon.org/search/?formInput=doctypes:supremecourt fromdate:1-2-1950 todate:28-2-1950&pagenum=0
Results found on page: 0
Processing March 1950 ...
Fetching: https://indiankanoon.org/search/?formInput=doctypes:supremecourt fromdate:1-3-1950 todate:31-3-1950&pagenum=0
Results found on page: 0
Processing April 1950 ...
Fetching: https://indiankanoon.org/search/?formInput=doctypes:supremecourt fromdate:1-4-1950 todate:30-4-1950&pagenum=0
Results found on page: 0
Processing May 1950 ...
Fetching: https://indiankanoon.org/search/?formInput=doctypes:supremecourt fromdate:1-5-1950 todate:31-5-1950&pagenum=0
Results found on page: 0
Processing June 1950 ...
Fetching: https://indiankanoon.org/search/?formInput=doctypes:supremecourt fromdate:1-6-1950 todate:30-6-1950&pagenum

KeyboardInterrupt: 

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import json

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}
base_url = 'https://indiankanoon.org'

# Step 1: Get court list
url = base_url + '/browse/'
page = requests.get(url, headers=headers)
soup = BeautifulSoup(page.content, 'html.parser')

courts = soup.find_all(class_='browselist')
if not courts:
    print("No courts found. Check HTML structure.")
    exit()

court = courts[0].find('a')  # Supreme Court of India
court_name = court.text
court_url = base_url + court['href']

# Step 2: Get years
page = requests.get(court_url, headers=headers)
soup = BeautifulSoup(page.content, 'html.parser')
years = soup.find_all(class_='browselist')

final_links = {court_name: []}

# Limiting to one year to test
for year_block in years[:1]:
    year_link = base_url + year_block.find('a')['href']
    year = year_block.find('a').text
    print(f"Scraping year: {year}")

    # Step 3: Get months
    page = requests.get(year_link, headers=headers)
    soup = BeautifulSoup(page.content, 'html.parser')
    months = soup.find_all(class_='browselist')

    for month_block in months[:1]:  # only one month
        month_url = base_url + month_block.find('a')['href']

        # Step 4: Loop through a few pages
        for p in range(3):
            time.sleep(2)
            paginated_url = f"{month_url}&pagenum={p}"
            page = requests.get(paginated_url, headers=headers)
            soup = BeautifulSoup(page.content, 'html.parser')
            results = soup.find_all(class_='result_url')

            if not results:
                print(f"No results on page {p}")
                break

            for result in results:
                case_link = base_url + result['href']
                final_links[court_name].append(case_link)

# Save to JSON
with open("test_links_Supreme_Court.json", "w") as f:
    json.dump(final_links, f, indent=4)

print("Saved links successfully.")


No courts found. Check HTML structure.


IndexError: list index out of range

: 

In [16]:
#used to scrape all month urls and judgement urls and save it into a .json file

all_judgment_links = []

for year, year_url in year_links:
    print(f"Processing year {year} ...")
    page = requests.get(year_url)
    soup = BeautifulSoup(page.content, 'html.parser')
    # Each month is a browselist
    for div in soup.find_all(class_='browselist'):
        for a in div.find_all('a'):
            month_url = base + a['href']
            # Now paginate through results
            for pagenum in range(100):  # Adjust as needed
                time.sleep(1)  # Be polite!
                paged_url = f"{month_url}&pagenum={pagenum}"
                r = requests.get(paged_url)
                s = BeautifulSoup(r.content, 'html.parser')
                #results = s.find_all(class_='result_url')
                #results = s.find_all('a', class_='result_title')
                results = s.select('a.result_title')
                if not results:
                    break  # No more results
                for result in results:
                    judgment_url = base + result['href']
                    all_judgment_links.append(judgment_url)
                print(f"Total judgments scraped: {len(all_judgment_links)}")
    print(f"Year {year} done.")

# Save to JSON
with open("supreme_court_judgments.json", "w") as f:
    json.dump(all_judgment_links, f, indent=2)
print(f"Total judgments scraped: {len(all_judgment_links)}")


Processing year 1950 ...
Year 1950 done.
Processing year 1951 ...
Year 1951 done.
Processing year 1952 ...
Year 1952 done.
Processing year 1953 ...
Year 1953 done.
Processing year 1954 ...
Year 1954 done.
Processing year 1955 ...
Year 1955 done.
Processing year 1956 ...
Year 1956 done.
Processing year 1957 ...
Year 1957 done.
Processing year 1958 ...


KeyboardInterrupt: 

In [17]:
import requests
from bs4 import BeautifulSoup
import time
import json

base = 'https://indiankanoon.org'
all_judgment_links = []

for year in range(2010, 2021):  # Example: 2010 to 2020
    print(f"Processing year {year} ...")
    # Construct the search URL for the whole year
    search_url = f"{base}/search/?formInput=doctypes:supremecourt fromdate:1-1-{year} todate:31-12-{year}"
    for pagenum in range(0, 100):  # Adjust range if needed
        url = f"{search_url}&pagenum={pagenum}"
        r = requests.get(url)
        soup = BeautifulSoup(r.content, 'html.parser')
        results = soup.find_all('a', class_='result_title')
        if not results:
            break  # No more results
        for result in results:
            judgment_url = base + result['href']
            all_judgment_links.append(judgment_url)
        time.sleep(1)  # Be polite!
    print(f"Year {year} done.")

# Save to JSON
with open("supreme_court_judgments.json", "w") as f:
    json.dump(all_judgment_links, f, indent=2)
print(f"Total judgments scraped: {len(all_judgment_links)}")


Processing year 2010 ...
Year 2010 done.
Processing year 2011 ...
Year 2011 done.
Processing year 2012 ...
Year 2012 done.
Processing year 2013 ...
Year 2013 done.
Processing year 2014 ...
Year 2014 done.
Processing year 2015 ...
Year 2015 done.
Processing year 2016 ...
Year 2016 done.
Processing year 2017 ...
Year 2017 done.
Processing year 2018 ...
Year 2018 done.
Processing year 2019 ...
Year 2019 done.
Processing year 2020 ...
Year 2020 done.
Total judgments scraped: 0


Now using the .json file to scrape the data and download it to save it in a file

In [ ]:
import os
import urllib
import urllib.request
import json
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import progressbar


path_of_Supreme_court_json_file = "links_Supreme_Court.json"

links_json = json.load(path_of_Supreme_court_json_file)
category = 'Supreme Court of India'
print("Started file ... {0} with docs = {1}\n\n\n".format(json_file,len(links_json[category])))
category_links = links_json[category]
year = json_file[len(json_file)-9:len(json_file)-5] # Obtain the name of year like '1947' or '2002'
ndocs = 0
os.mkdir('Yearwise_data/'+ year)

while(len(category_links)>0):
  time.sleep(2) 
  links_done_in_this_loop = []
  for i in progressbar.progressbar(range(len(category_links))):
      BASE_URL = category_links[i]
      try:
          html = urllib.request.urlopen(BASE_URL).read()
      except urllib.error.HTTPError:
          print("Ocuured at doc",ndocs)
          time.sleep(2)

      else:
          soup = BeautifulSoup(html, "lxml")
          data_html = soup.find("div", attrs={"class": "judgments"})
          text = data_html.get_text()
          path =  'Yearwise_data/' + year + '/' + year +'_' +str(ndocs) # Path where the yearwise data file will save
          f= open(path,"w+")
          f.write(text)
          ndocs = ndocs+1
          links_done_in_this_loop.append(BASE_URL)
          if(ndocs%100==0):
              time.sleep(2)          # If number of documents downloaded is 100 then go for sleep for 2 sec
  for link in links_done_in_this_loop:
      category_links.remove(link)
  print("Docs that were downloaded: {0}\n\n\n".format(ndocs))
